In [17]:
# Importing the requried packages
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
import torch.nn.utils.prune as prune
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition, mark_inset)
import matplotlib
import pandas as pd 


from qbo1d import adsolver, utils
import netCDF4 as nc


import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib
import netCDF4 as nc
import scipy.stats as st
import scipy

In [18]:
# for scaling input or output

class GlobalScaler():
    def __init__(self, X):
        self.abs_max = X.std().max() #(0)
        
    def transform(self, X):
        return X / self.abs_max
    
    def inverse_transform(self, X):
        return X * self.abs_max

In [19]:
# Set the resolution of the 1D model
rez=125

In [ ]:
solver = adsolver.ADSolver(z_min=17e3, z_max=35e3, dz=rez, t_min=0.0, t_max=365*100*86400, dt=86400.0, w =0.0, kappa=3e-1)

In [20]:
# loading the training data

data = xr.open_dataset('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/QBO_0.5x_100yrs_' + str(rez) + 'm.nc')
u = data.u[:-1, 1:-1].values
f = data.f[1:, 1:-1].values

trainset = TensorDataset(torch.from_numpy(u[:90*365]).float(),torch.from_numpy(f[:90*365]).float())
valset = TensorDataset(torch.from_numpy(u[90*365:]).float(),torch.from_numpy(f[90*365:]).float())

In [23]:
scaler_Y = GlobalScaler(trainset[:][1])
print('scaler_Y:' + str(scaler_Y.abs_max))

scaler_Y:tensor(3.2622e-06)


In [24]:
train_dataloader = DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=4)
val_dataloader  = DataLoader(valset, batch_size= len(valset), shuffle=True, num_workers=4)

In [25]:
# loading the testing data

data = xr.open_dataset('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/QBO_0.5x_1000yrs_'+ str(rez) + 'm.nc')
u = data.u[:-1, 1:-1].values
f = data.f[1:, 1:-1].values

# u
testset = TensorDataset(torch.from_numpy(u).float(),torch.from_numpy(f).float())
test_dataloader  = DataLoader(testset, batch_size= len(testset), shuffle=False, num_workers=4)

In [27]:
actv = nn.Tanh()

In [ ]:
# Different configurations are used for training.

In [19]:
configs = {'1': {'nl':9, 'nch':11, 'ks':17, 'dil':1},
           '2': {'nl':9, 'nch':11, 'ks':19, 'dil':1},
           '3': {'nl':9, 'nch':10, 'ks':21, 'dil':1},
           '4': {'nl':9, 'nch':9, 'ks':23, 'dil':1},
           '5': {'nl':9, 'nch':9, 'ks':25, 'dil':1},
           
           '6': {'nl':10, 'nch':11, 'ks':17, 'dil':1},
           '7': {'nl':11, 'nch':10, 'ks':17, 'dil':1},
           '8': {'nl':12, 'nch':9, 'ks':17, 'dil':1},
           '9': {'nl':13, 'nch':9, 'ks':17, 'dil':1},
           
           '10': {'nl':8, 'nch':16, 'ks':9, 'dil':2},
           '11': {'nl':8, 'nch':16, 'ks':9, 'dil':3},
           }

In [20]:
configs = {'1': {'nl':14, 'nch':10, 'ks':13, 'dil':1},
           '2': {'nl':15, 'nch':9, 'ks':13, 'dil':1}}

In [21]:
configs = {'100': {'1': {'nl':8, 'nch':16, 'ks':9, 'dil':2},
                   '2': {'nl':9, 'nch':13, 'ks':13, 'dil':2}},
           '125':{'1': {'nl':9, 'nch':14, 'ks':11, 'dil':2}},
           '150':{'1': {'nl':8, 'nch':15, 'ks':11, 'dil':2}},
           '200':{'1': {'nl':6, 'nch':18, 'ks':11, 'dil':2}},
           '300':{'1': {'nl':5, 'nch':23, 'ks':9, 'dil':2}},
           '400':{'1': {'nl':4, 'nch':28, 'ks':9, 'dil':2}},
           '600':{'1': {'nl':4, 'nch':32, 'ks':7, 'dil':2}},
           '750':{'1': {'nl':4, 'nch':38, 'ks':5, 'dil':2}},
           '900':{'1': {'nl':4, 'nch':38, 'ks':5, 'dil':2}},
           '1200':{'1': {'nl':3, 'nch':54, 'ks':5, 'dil':2}}}

In [22]:
configs = {'600': {'1': {'nl':4, 'nch':28, 'ks':9, 'dil':1},
                   '2': {'nl':8, 'nch':33, 'ks':5, 'dil':1}}}

In [27]:
configs = {'100':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '125':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '150':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '200':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '250':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '300':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '400':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '600':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '750':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '900':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '1200':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}},
           '1500':{'1': {'nl':4, 'nch':19, 'ks':19, 'dil':1}}}

In [12]:
configs = {'100': {'1': {'nl':9, 'nch':8, 'ks':29, 'dil':1}}}

In [12]:
configs = {'360': {'1': {'nl':5, 'nch':31, 'ks':5, 'dil':1},
                   '2': {'nl':5, 'nch':23, 'ks':9, 'dil':1},
                   '3': {'nl':5, 'nch':19, 'ks':13, 'dil':1},
                   '4': {'nl':5, 'nch':17, 'ks':17, 'dil':1},
                   '5': {'nl':5, 'nch':15, 'ks':21, 'dil':1},
                   
                   '6': {'nl':6, 'nch':20, 'ks':9, 'dil':1},
                   '7': {'nl':7, 'nch':18, 'ks':9, 'dil':1},
                   '8': {'nl':8, 'nch':16, 'ks':9, 'dil':1},
                   '9': {'nl':9, 'nch':15, 'ks':9, 'dil':1},
                   '10': {'nl':10, 'nch':14, 'ks':9, 'dil':1},
                   
                   '11': {'nl':5, 'nch':26, 'ks':7, 'dil':2},
                   '12': {'nl':5, 'nch':26, 'ks':7, 'dil':3},
                   '13': {'nl':4, 'nch':32, 'ks':7, 'dil':2}}}

In [28]:
configs = {'125': {'1': {'nl':5, 'nch':18, 'ks':15, 'dil':2},
                   '2': {'nl':6, 'nch':16, 'ks':15, 'dil':2},
                   '3': {'nl':7, 'nch':14, 'ks':15, 'dil':2},
                   '4': {'nl':8, 'nch':13, 'ks':15, 'dil':2},
                   '5': {'nl':9, 'nch':12, 'ks':15, 'dil':2}}}

In [29]:
# The CNN for interpretability
class FullyConnected(nn.Module):
    def __init__(self, nl, nch, ks, dil):
        super(FullyConnected, self).__init__()

        self.linear_stack = nn.ModuleList()

        self.linear_stack.append(nn.Conv1d(in_channels=1, out_channels=nch, kernel_size=ks, dilation=dil, padding="same"))
        self.linear_stack.append(actv)

        for i in range(nl-2):
            self.linear_stack.append(nn.Conv1d(in_channels=nch, out_channels=nch, kernel_size=ks, dilation=dil, padding="same"))
            self.linear_stack.append(actv)
            
        self.linear_stack.append(nn.Conv1d(in_channels=nch, out_channels=1, kernel_size=ks, dilation=dil, padding="same"))

    def forward(self, X):
        x = X[:, None]
        for layer in self.linear_stack:
            x = layer(x)
        return x.squeeze()

In [30]:
class EarlyStopper:
    def __init__(self, nl, nch, ks, dil, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            
            # save model
            torch.save(model.state_dict(), '/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/NNs/Model/CNN_' + 
                       str(nl) + 'L_' + str(nch) + 'C_k' + str(ks) + '_D' + str(dil) + '_0.5x_' + str(rez) + 'm.pth')
            
            

        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [31]:
# training loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    avg_loss = 0
    for batch, (X, Y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.to(device).float())
        loss = loss_fn(pred.float(), scaler_Y.transform(Y.to(device)).float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   
        with torch.no_grad():
            avg_loss += loss.item()
            
    avg_loss /= len(dataloader)
    
    return avg_loss


# validating loop
def val_loop(dataloader, model, loss_fn):
    avg_loss = 0
    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            # Compute prediction and loss
            pred = model(X.to(device).float())
            loss = loss_fn(pred.float(), scaler_Y.transform(Y.to(device)).float())
            avg_loss += loss.item()
            
    avg_loss /= len(dataloader)
    
    return avg_loss

In [32]:
for rez in [125]: #100, 125, 150, 200, 250, 300, 400, 600, 750, 900, 
    # for case in range(len(configs[str(rez)])):
    for case in range(4, 5):

        nl = configs[str(rez)][str(case+1)]['nl']
        nch = configs[str(rez)][str(case+1)]['nch']
        ks = configs[str(rez)][str(case+1)]['ks']
        dil = configs[str(rez)][str(case+1)]['dil']
    
        
        model = (FullyConnected(nl, nch, ks, dil)).float()
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda"
            if torch.cuda.device_count() > 1:
                model = nn.DataParallel(model)
        print(device)
        model.to(device)
    
        #nparams
        print('case' + str(case+1))
        print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    
        data = xr.open_dataset('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/QBO_0.5x_100yrs_' + str(rez) + 'm.nc')
        u = data.u[:-1, 1:-1].values
        f = data.f[1:, 1:-1].values
        
        #big data
        trainset = TensorDataset(torch.from_numpy(u[:90*365]).float(),torch.from_numpy(f[:90*365]).float())
        valset = TensorDataset(torch.from_numpy(u[90*365:]).float(),torch.from_numpy(f[90*365:]).float())
        
        scaler_Y = GlobalScaler(trainset[:][1])
        
        train_dataloader = DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=4)
        val_dataloader  = DataLoader(valset, batch_size= len(valset), shuffle=True, num_workers=4)
        
        data = xr.open_dataset('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/QBO_0.5x_1000yrs_'+ str(rez) + 'm.nc')
        u = data.u[:-1, 1:-1].values
        f = data.f[1:, 1:-1].values
        
        # u
        testset = TensorDataset(torch.from_numpy(u).float(),torch.from_numpy(f).float())
        test_dataloader  = DataLoader(testset, batch_size= len(testset), shuffle=False, num_workers=4)
        
        train_losses = []
        val_losses = []
        learning_rate = 1e-2
        epochs = 6000
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        scaler_Y = GlobalScaler(trainset[:][1])
        print('scaler_Y:' + str(scaler_Y.abs_max))
    
        # training
        k = 0
        early_stopper = EarlyStopper(nl, nch, ks, dil, patience=30, min_delta=0.0)
        for t in range(epochs):
            train_loss = train_loop(train_dataloader, model, nn.MSELoss(), optimizer)
            train_losses.append(train_loss)
            val_loss = val_loop(val_dataloader, model, nn.MSELoss())
            val_losses.append(val_loss)
            if t % 100 ==0:
                print(f"Epoch {t+1}\n-------------------------------")
                print(val_loss)
                # print(train_loss)
            if early_stopper.early_stop(val_loss):
                if k <8:
                    early_stopper = EarlyStopper(nl, nch, ks, dil, patience=30, min_delta=0.0)
                    learning_rate = learning_rate * 0.25
                    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    k += 1
                    print("New Learning Rate: " + str(learning_rate))
                else:
                    break
        print("Done!")
    
        model.load_state_dict(torch.load('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/NNs/Model/CNN_' + 
                           str(nl) + 'L_' + str(nch) + 'C_k' + str(ks) + '_D' + str(dil) + '_0.5x_' + str(rez) + 'm.pth'))
    
        with torch.no_grad():
            for batch, (X, Y) in enumerate(test_dataloader):
                pred = scaler_Y.inverse_transform(model(X))
    
        f_array = xr.DataArray(pred.cpu(), dims=('time', 'height'))
        dataset = xr.Dataset({'f': (['time', 'height'], f_array.data, [('units', 'ms-2')])})
        dataset.to_netcdf('/glade/derecho/scratch/pahlavan/qbo1d/Nonlocality-0.5x/NNs/Data/CNN_' + 
                           str(nl) + 'L_' + str(nch) + 'C_k' + str(ks) + '_D' + str(dil) + '_0.5x_' + str(rez) + 'm_offline.nc')
        print((((pred - (test_dataloader.dataset[:][1]).to(device))**2).mean())**0.5)
        print((np.corrcoef(pred.cpu().flatten(), (test_dataloader.dataset[:][1]).cpu().flatten())[0][1])**2)
        print('###############')

cpu
case5
15577
scaler_Y:tensor(3.2622e-06)
Epoch 1
-------------------------------
0.25814035534858704
Epoch 101
-------------------------------
0.024000143632292747
Epoch 201
-------------------------------
0.007515815086662769
New Learning Rate: 0.0025
Epoch 301
-------------------------------
0.0060591562651097775
Epoch 401
-------------------------------
0.0030676990281790495
New Learning Rate: 0.000625
Epoch 501
-------------------------------
0.0018914799438789487
Epoch 601
-------------------------------
0.001767899258993566
Epoch 701
-------------------------------
0.0016886385856196284
New Learning Rate: 0.00015625
Epoch 801
-------------------------------
0.001522848499007523
Epoch 901
-------------------------------
0.0014905155403539538
New Learning Rate: 3.90625e-05
Epoch 1001
-------------------------------
0.0014578084228560328
Epoch 1101
-------------------------------
0.001447984715923667
New Learning Rate: 9.765625e-06
New Learning Rate: 2.44140625e-06
New Learning R